# Aggregration e Grouping

Uma parte essencial da análise de dados grandes é a sumarização eficiente: agregações de computação como `sum()`, `median()`, `mean()`, `min()` e `max()`, nas quais um único número fornece informações sobre a natureza de um potencialmente grande conjunto de dados. Neste notebook, exploraremos agregações no Pandas, desde operações simples semelhantes às que vimos nas matrizes NumPy, até operações mais sofisticadas baseadas no conceito de groupby.



In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Planets Data

Aqui usaremos o conjunto de dados Planetas, disponível no pacote [Seaborn](http://seaborn.pydata.org/). Ele fornece informações sobre planetas que os astrônomos descobriram em torno de outras estrelas (conhecidas como planetas extra-solares ou exoplanetas). Pode ser baixado com um simples comando Seaborn:

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Isso tem alguns detalhes sobre os mais de 1.000 planetas extra-solares descobertos até 2014.

## Agregação simples com pandas

In [4]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [5]:
ser.sum()

2.811925491708157

In [6]:
ser.mean()

0.5623850983416314

Para um `DataFrame`, por padrão, as agregações retornam resultados dentro de cada coluna:

In [7]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [8]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Ao especificar o argumento do eixo, você pode agregar em cada linha:

In [9]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas `Serie` e os `DataFrames` incluem todos os agregados comuns mencionados em Agregações: Mín, Máx e Tudo o Mais; Além disso, existe um método de conveniência `describe()` que calcula vários agregados comuns para cada coluna e retorna o resultado. Vamos usar isso nos dados de Planetas, por agora descartar linhas com valores ausentes:

In [10]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Essa pode ser uma maneira útil de começar a entender as propriedades gerais de um conjunto de dados. Por exemplo, vemos na coluna do `year`  que, embora exoplanets tenham sido descobertos em 1989, metade de todos os expolanets conhecidos não foram descobertos até 2010 ou depois. Isso se deve em grande parte à missão Kepler, que é um telescópio espacial projetado especificamente para encontrar planetas eclipsantes em torno de outras estrelas.

A tabela a seguir resume algumas outras agregações internas do Pandas:

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |


Estes são todos os métodos dos objetos `` DataFrame`` e `` Series``.

## GroupBy: Split, Apply, Combine

Agregações simples podem fornecer uma amostra do seu conjunto de dados, mas geralmente preferimos agregar condicionalmente em algum rótulo ou índice: isso é implementado na chamada operação groupby. O nome "agrupar por" vem de um comando na linguagem do banco de dados SQL, mas talvez seja mais esclarecedor pensar nos termos criados por Hadley Wickham, da Rstats fama: dividir, aplicar, combinar.

### Split, apply, combine

Um exemplo canônico desta operação de divisão de aplicar e combinar, em que "aplicar" é uma agregação de soma.

Isso deixa claro o que o grupo realiza:

- A etapa de *split* envolve dividir e agrupar um DataFrame, dependendo do valor da chave especificada.
- A etapa de *apply* envolve o cálculo de alguma função, geralmente uma agregação, transformação ou filtragem, dentro dos grupos individuais.
- A etapa de *combine* mescla os resultados dessas operações em uma matriz de saída.

Embora isso certamente possa ser feito manualmente usando alguma combinação dos comandos de mascaramento, agregação e mesclagem abordados anteriormente, uma importante conclusão é que as divisões intermediárias não precisam ser instanciadas explicitamente. Em vez disso, o GroupBy pode (geralmente) fazer isso em uma única passagem sobre os dados, atualizando a soma, média, contagem, min ou outro agregado para cada grupo ao longo do caminho. O poder do GroupBy é que ele abstrai essas etapas: o usuário não precisa pensar em como o cálculo é realizado, mas pensa na operação como um todo.

Como um exemplo concreto, vamos dar uma olhada no Pandas para o cálculo mostrado neste diagrama. Vamos começar criando a entrada DataFrame:

In [11]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


A operação mais básica de divisão de aplicação e combinação pode ser calculada com o método `groupby()` de DataFrames, passando o nome da coluna de chave desejada:

In [12]:
df.groupby('key')

Observe que o que é retornado não é um conjunto de `DataFrames`, mas um objeto `DataFrameGroupBy`. Este objeto é onde está a mágica: você pode pensar nele como uma visão especial do `DataFrame`, que está pronto para cavar nos grupos, mas não faz cálculos reais até que a agregação seja aplicada. Essa abordagem de "avaliação lenta" significa que agregados comuns podem ser implementados com muita eficiência de uma maneira quase transparente para o usuário.

Para produzir um resultado, podemos aplicar uma agregação a esse objeto `DataFrameGroupBy`, que executará as etapas de aplicar/combinar apropriadas para produzir o resultado desejado:

In [13]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


O método `sum()` é apenas uma possibilidade aqui; você pode aplicar praticamente qualquer função de agregação comum do Pandas ou NumPy, bem como praticamente qualquer operação válida do `DataFrame`, como veremos na discussão a seguir.

## O GroupBy object

O objeto GroupBy é uma abstração muito flexível. De várias maneiras, você pode simplesmente tratá-lo como se fosse uma coleção de DataFrames, e isso faz as coisas difíceis por trás. Vamos ver alguns exemplos usando os dados do Planetas.

### column indexing

O objeto `GroupBy` oferece suporte à indexação de colunas da mesma maneira que o `DataFrame` e retorna um objeto `GroupBy` modificado. Por exemplo:

In [14]:
planets.groupby('method')

In [15]:
planets.groupby('method')['orbital_period']

Aqui, selecionamos um grupo de séries específico do grupo `DataFrame` original por referência ao nome da coluna. Como no objeto `GroupBy`, nenhum cálculo é feito até chamarmos alguma agregação no objeto:

In [16]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Isso dá uma idéia da escala geral dos períodos orbitais (em dias) aos quais cada método é sensível.

### Iteration groups

O objeto `GroupBy` suporta iteração direta sobre os grupos, retornando cada grupo como um `Series` ou `DataFrame`:

In [17]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


### Métodos de expedição

Por meio de alguma mágica da classe Python, qualquer método não implementado explicitamente pelo objeto GroupBy será passado e chamado pelos grupos, sejam eles `DataFrame` ou `Series`. Por exemplo, você pode usar o método `describe()` de `DataFrames` para executar um conjunto de agregações que descrevem cada grupo nos dados:

In [18]:
planets.groupby('method')['year'].describe().unstack()

       method                       
count  Astrometry                          2.000000
       Eclipse Timing Variations           9.000000
       Imaging                            38.000000
       Microlensing                       23.000000
       Orbital Brightness Modulation       3.000000
       Pulsar Timing                       5.000000
       Pulsation Timing Variations         1.000000
       Radial Velocity                   553.000000
       Transit                           397.000000
       Transit Timing Variations           4.000000
mean   Astrometry                       2011.500000
       Eclipse Timing Variations        2010.000000
       Imaging                          2009.131579
       Microlensing                     2009.782609
       Orbital Brightness Modulation    2011.666667
       Pulsar Timing                    1998.400000
       Pulsation Timing Variations      2007.000000
       Radial Velocity                  2007.518987
       Transit             

### Aggregate, filter, transform, apply

A discussão anterior focou na agregação para a operação de combinação, mas há mais opções disponíveis. Em particular, os objetos GroupBy têm métodos `agregate()`, `filter()`, `transform()` e `apply()` que implementam com eficiência uma variedade de operações úteis antes de combinar os dados agrupados.

In [19]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


### Aggregation

Agora estamos familiarizados com agregações `GroupBy` com `sum()`, `median()` e similares, mas o método `agregate()` permite ainda mais flexibilidade. Pode pegar uma string, uma função ou uma lista dela e calcular todos os agregados de uma só vez. Aqui está um exemplo rápido combinando tudo isso:

In [20]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Outro padrão útil é passar um nome de coluna de mapeamento de dicionário para operações a serem aplicadas nessa coluna: 

In [22]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


### Filtering

Uma operação de filtragem permite eliminar dados com base nas propriedades do grupo. Por exemplo, convém manter todos os grupos nos quais o desvio padrão é maior que algum valor crítico:

In [23]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

A função de filtro deve retornar um valor booleano especificando se o grupo passa na filtragem. Aqui, como o grupo A não possui um desvio padrão maior que 4, ele é descartado do resultado.

### TRansformation

Enquanto a agregação deve retornar uma versão reduzida dos dados, a transformação pode retornar uma versão transformada dos dados completos para recombinar. Para essa transformação, a saída tem a mesma forma que a entrada. Um exemplo comum é centralizar os dados subtraindo a média do grupo:

In [24]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


### Método apply()

O método `apply()` permite aplicar uma função arbitrária aos resultados do grupo. A função deve pegar um DataFrame e retornar um objeto Pandas (por exemplo, DataFrame, Series) ou um escalar; a operação de combinação será adaptada ao tipo de saída retornada.

Por exemplo, aqui está um `apply()` que normaliza a primeira coluna pela soma da segunda:

In [25]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


### Grouping exemplo

Como exemplo disso, em algumas linhas do código Python podemos juntar tudo isso e contar planetas descobertos por método e por década:



In [26]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


Isso mostra o poder de combinar muitas das operações que discutimos até o momento ao analisar conjuntos de dados realistas. Imediatamente adquirimos uma compreensão grosseira de quando e como os planetas foram descobertos nas últimas décadas!